In [2]:


%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
# Import necessary libraries
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd

In [3]:
%%R

# My commonly used R imports

# Install tidygeocoder if it is not already installed
if (!require(tidygeocoder)) {
  install.packages("tidygeocoder")
}
library(tidygeocoder)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidygeocoder


## Aquarion Water Co of CT-Main System

In [3]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Aquarion Water Co of CT-Main System.csv")

# Preview the data to check the column names and structure
print(head(data))


Rows: 3974 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): LOCATION IDENTIFIER, STREET ADDRESS, TOWN, merged address, STATE, ...
dbl  (1): SITE ID
lgl  (3): STREET NUMBER, STREET NAME, is_match

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 15
  `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS`  TOWN  `merged address` STATE
      <dbl> <chr>                 <chr>             <chr> <chr>            <chr>
1 700000246 752 PARK AVE          752 PARK AVE      BRID… 752 PARK AVE BR… CT   
2 700001555 745 HANCOCK AVE       745 HANCOCK AVE   BRID… 745 HANCOCK AVE… CT   
3 700001611 725 PARK AVE          725 PARK AVE      BRID… 725 PARK AVE BR… CT   
4 700001683 25 PRINCETON ST       25 PRINCETON ST   BRID… 25 PRINCETON ST… CT   
5 700001756 459 IRANISTAN AVE     459 IRANISTAN AVE BRID… 459 IRANISTAN A… C

In [4]:
%%R

colnames(data)
# [1] "SITE ID" "LOCATION IDENTIFIER" "STREET ADDRESS" "TOWN" "merged address" "STATE" ...



 [1] "SITE ID"                                    
 [2] "LOCATION IDENTIFIER"                        
 [3] "STREET ADDRESS"                             
 [4] "TOWN"                                       
 [5] "merged address"                             
 [6] "STATE"                                      
 [7] "ENTIRE SERVICE LINE MATERIAL CLASSIFICATION"
 [8] "BUILDING TYPE"                              
 [9] "SOURCE FILE"                                
[10] "STREET NUMBER"                              
[11] "STREET NAME"                                
[12] "extracted_pswid"                            
[13] "is_match"                                   
[14] "PWSID"                                      
[15] "SYSTEM NAME"                                


In [5]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)


# A tibble: 6 × 17
  `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS`  TOWN  `merged address` STATE
      <dbl> <chr>                 <chr>             <chr> <chr>            <chr>
1 700000246 752 PARK AVE          752 PARK AVE      BRID… 752 PARK AVE BR… CT   
2 700001555 745 HANCOCK AVE       745 HANCOCK AVE   BRID… 745 HANCOCK AVE… CT   
3 700001611 725 PARK AVE          725 PARK AVE      BRID… 725 PARK AVE BR… CT   
4 700001683 25 PRINCETON ST       25 PRINCETON ST   BRID… 25 PRINCETON ST… CT   
5 700001756 459 IRANISTAN AVE     459 IRANISTAN AVE BRID… 459 IRANISTAN A… CT   
6 700001878 95 JONES AVE          95 JONES AVE      BRID… 95 JONES AVE BR… CT   
# ℹ 11 more variables: `ENTIRE SERVICE LINE MATERIAL CLASSIFICATION` <chr>,
#   `BUILDING TYPE` <chr>, `SOURCE FILE` <chr>, `STREET NUMBER` <lgl>,
#   `STREET NAME` <lgl>, extracted_pswid <chr>, is_match <lgl>, PWSID <chr>,
#   `SYSTEM NAME` <chr>, lat <dbl>, long <dbl>


Passing 3,944 addresses to the Nominatim single address geocoder


In [6]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)


# A tibble: 3,974 × 17
   `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS` TOWN  `merged address` STATE
       <dbl> <chr>                 <chr>            <chr> <chr>            <chr>
 1 700000246 752 PARK AVE          752 PARK AVE     BRID… 752 PARK AVE BR… CT   
 2 700001555 745 HANCOCK AVE       745 HANCOCK AVE  BRID… 745 HANCOCK AVE… CT   
 3 700001611 725 PARK AVE          725 PARK AVE     BRID… 725 PARK AVE BR… CT   
 4 700001683 25 PRINCETON ST       25 PRINCETON ST  BRID… 25 PRINCETON ST… CT   
 5 700001756 459 IRANISTAN AVE     459 IRANISTAN A… BRID… 459 IRANISTAN A… CT   
 6 700001878 95 JONES AVE          95 JONES AVE     BRID… 95 JONES AVE BR… CT   
 7 700002813 241 PINE ST           241 PINE ST      BRID… 241 PINE ST BRI… CT   
 8 700002952 24 WALLACE ST         24 WALLACE ST    BRID… 24 WALLACE ST B… CT   
 9 700003044 3101 MAIN ST          3101 MAIN ST     BRID… 3101 MAIN ST BR… CT   
10 700003374 522 FAIRFIELD AVE     522 FAIRFIELD A… BRID… 522 FAIRFIELD A… CT   
# ℹ 3

In [7]:
%%R

write_csv(data_geocoded, "Aquarion_Water_Co_geocoded.csv")


## Windham Water Works

In [8]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Windham Water Works.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 329 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): LOCATION IDENTIFIER, STREET ADDRESS, TOWN, merged address, STATE, ...
dbl  (1): SITE ID
lgl  (3): STREET NUMBER, STREET NAME, is_match

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 15
  `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS` TOWN   `merged address` STATE
      <dbl> <chr>                 <chr>            <chr>  <chr>            <chr>
1     40418 41.708, -72.206       27 Park St       Windh… 27 Park St Wind… CT   
2     30329 41.706, -72.2         217 Windham Rd   Windh… 217 Windham Rd … CT   
3     20521 41.718, -72.213       118 Chestnut St  Windh… 118 Chestnut St… CT   
4     20959 41.717, -72.215       150 Walnut St    Windh… 150 Walnut St W… CT   
5     20317 41.716, -72.193       348 Bricktop Rd  Windh… 348 Bricktop Rd… CT

In [9]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

# A tibble: 6 × 17
  `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS` TOWN   `merged address` STATE
      <dbl> <chr>                 <chr>            <chr>  <chr>            <chr>
1     40418 41.708, -72.206       27 Park St       Windh… 27 Park St Wind… CT   
2     30329 41.706, -72.2         217 Windham Rd   Windh… 217 Windham Rd … CT   
3     20521 41.718, -72.213       118 Chestnut St  Windh… 118 Chestnut St… CT   
4     20959 41.717, -72.215       150 Walnut St    Windh… 150 Walnut St W… CT   
5     20317 41.716, -72.193       348 Bricktop Rd  Windh… 348 Bricktop Rd… CT   
6     31053 41.7, -72.21          327 South St     Windh… 327 South St Wi… CT   
# ℹ 11 more variables: `ENTIRE SERVICE LINE MATERIAL CLASSIFICATION` <chr>,
#   `BUILDING TYPE` <chr>, `SOURCE FILE` <chr>, `STREET NUMBER` <lgl>,
#   `STREET NAME` <lgl>, extracted_pswid <chr>, is_match <lgl>, PWSID <chr>,
#   `SYSTEM NAME` <chr>, lat <dbl>, long <dbl>


Passing 329 addresses to the Nominatim single address geocoder


In [10]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)

# A tibble: 329 × 17
   `SITE ID` `LOCATION IDENTIFIER` `STREET ADDRESS` TOWN  `merged address` STATE
       <dbl> <chr>                 <chr>            <chr> <chr>            <chr>
 1     40418 41.708, -72.206       27 Park St       Wind… 27 Park St Wind… CT   
 2     30329 41.706, -72.2         217 Windham Rd   Wind… 217 Windham Rd … CT   
 3     20521 41.718, -72.213       118 Chestnut St  Wind… 118 Chestnut St… CT   
 4     20959 41.717, -72.215       150 Walnut St    Wind… 150 Walnut St W… CT   
 5     20317 41.716, -72.193       348 Bricktop Rd  Wind… 348 Bricktop Rd… CT   
 6     31053 41.7, -72.21          327 South St     Wind… 327 South St Wi… CT   
 7     10069 41.72, -72.201        169 Ash St.      Wind… 169 Ash St. Win… CT   
 8     20009 41.713, -72.203       263 Main St      Wind… 263 Main St Win… CT   
 9     31334 41.707, -72.208       190 S Park St    Wind… 190 S Park St W… CT   
10     30813 41.709, -72.2         30 Potter St     Wind… 30 Potter St Wi… CT   
# ℹ 319

In [11]:
%%R

write_csv(data_geocoded, "Windham_Water_Works_geocoded.csv")

## Waterbury Water Department

In [4]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Waterbury Water Department.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 952 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): STREET ADDRESS, TOWN, merged address, STATE, ENTIRE SERVICE LINE MA...
dbl (1): SITE ID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 9
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
      <dbl> <chr>            <chr> <chr>            <chr> <chr>                 
1    102148 120 EASTON AVE   Wate… 120 EASTON AVE … Conn… Lead                  
2    102618 107 PLATT ST     Wate… 107 PLATT ST Wa… Conn… Lead                  
3    103178 1089 NORTH MAIN… Wate… 1089 NORTH MAIN… Conn… Lead                  
4    103188 1063 NORTH MAIN… Wate… 1063 NORTH MAIN… Conn… Lead                  
5    103208 1043 NORTH MAIN… Wate… 1043 NORTH MAIN… Conn… Lead                  
6    103218 1027 NORTH MAIN… Wate… 1027 NORTH 

In [5]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

# A tibble: 6 × 11
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
      <dbl> <chr>            <chr> <chr>            <chr> <chr>                 
1    102148 120 EASTON AVE   Wate… 120 EASTON AVE … Conn… Lead                  
2    102618 107 PLATT ST     Wate… 107 PLATT ST Wa… Conn… Lead                  
3    103178 1089 NORTH MAIN… Wate… 1089 NORTH MAIN… Conn… Lead                  
4    103188 1063 NORTH MAIN… Wate… 1063 NORTH MAIN… Conn… Lead                  
5    103208 1043 NORTH MAIN… Wate… 1043 NORTH MAIN… Conn… Lead                  
6    103218 1027 NORTH MAIN… Wate… 1027 NORTH MAIN… Conn… Lead                  
# ℹ abbreviated name: ¹​`ENTIRE SERVICE LINE MATERIAL CLASSIFICATION`
# ℹ 5 more variables: `BUILDING TYPE` <chr>, PWSID <chr>, `SYSTEM NAME` <chr>,
#   lat <dbl>, long <dbl>


Passing 852 addresses to the Nominatim single address geocoder


In [6]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)

# A tibble: 952 × 11
   `SITE ID` `STREET ADDRESS`   TOWN      `merged address`                 STATE
       <dbl> <chr>              <chr>     <chr>                            <chr>
 1    102148 120 EASTON AVE     Waterbury 120 EASTON AVE Waterbury Connec… Conn…
 2    102618 107 PLATT ST       Waterbury 107 PLATT ST Waterbury Connecti… Conn…
 3    103178 1089 NORTH MAIN ST Waterbury 1089 NORTH MAIN ST Waterbury Co… Conn…
 4    103188 1063 NORTH MAIN ST Waterbury 1063 NORTH MAIN ST Waterbury Co… Conn…
 5    103208 1043 NORTH MAIN ST Waterbury 1043 NORTH MAIN ST Waterbury Co… Conn…
 6    103218 1027 NORTH MAIN ST Waterbury 1027 NORTH MAIN ST Waterbury Co… Conn…
 7    103368 212 EASTON AVE     Waterbury 212 EASTON AVE Waterbury Connec… Conn…
 8    103428 213 EASTON AVE     Waterbury 213 EASTON AVE Waterbury Connec… Conn…
 9    103478 1208 NORTH MAIN ST Waterbury 1208 NORTH MAIN ST Waterbury Co… Conn…
10    103488 1214 NORTH MAIN ST Waterbury 1214 NORTH MAIN ST Waterbury Co… Conn…
# ℹ 942

In [7]:
%%R

write_csv(data_geocoded, "Waterbury_Water_Department_geocoded.csv")

## New London Dept. of Public Utilities

In [9]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("New London Dept. of Public Utilities.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 526 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): SITE ID, STREET ADDRESS, TOWN, merged address, STATE, ENTIRE SERVIC...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 9
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
  <chr>     <chr>            <chr> <chr>            <chr> <chr>                 
1 6 - 40620 1 COLVER ST      New … 1 COLVER ST New… CT    Lead                  
2 63 - 383… 10 Lyman Ct      New … 10 Lyman Ct New… CT    Lead                  
3 75 - 385… 10 ROSEMARY ST   New … 10 ROSEMARY ST … CT    Lead                  
4 94 - 388… 100 SCHOOL ST    New … 100 SCHOOL ST N… CT    Lead                  
5 99 - 406… 1000 OCEAN AVE   New … 1000 OCEAN AVE … CT    Lead                  
6 100 - 41… 1001 OCEAN AVE   New … 1001 OCEAN AVE … CT    Lead 

In [10]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

# A tibble: 6 × 11
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
  <chr>     <chr>            <chr> <chr>            <chr> <chr>                 
1 6 - 40620 1 COLVER ST      New … 1 COLVER ST New… CT    Lead                  
2 63 - 383… 10 Lyman Ct      New … 10 Lyman Ct New… CT    Lead                  
3 75 - 385… 10 ROSEMARY ST   New … 10 ROSEMARY ST … CT    Lead                  
4 94 - 388… 100 SCHOOL ST    New … 100 SCHOOL ST N… CT    Lead                  
5 99 - 406… 1000 OCEAN AVE   New … 1000 OCEAN AVE … CT    Lead                  
6 100 - 41… 1001 OCEAN AVE   New … 1001 OCEAN AVE … CT    Lead                  
# ℹ abbreviated name: ¹​`ENTIRE SERVICE LINE MATERIAL CLASSIFICATION`
# ℹ 5 more variables: `BUILDING TYPE` <chr>, PWSID <chr>, `SYSTEM NAME` <chr>,
#   lat <dbl>, long <dbl>


Passing 526 addresses to the Nominatim single address geocoder


In [11]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)

# A tibble: 526 × 11
   `SITE ID`   `STREET ADDRESS` TOWN       `merged address`              STATE
   <chr>       <chr>            <chr>      <chr>                         <chr>
 1 6 - 40620   1 COLVER ST      New London 1 COLVER ST New London CT     CT   
 2 63 - 38321  10 Lyman Ct      New London 10 Lyman Ct New London CT     CT   
 3 75 - 38501  10 ROSEMARY ST   New London 10 ROSEMARY ST New London CT  CT   
 4 94 - 38859  100 SCHOOL ST    New London 100 SCHOOL ST New London CT   CT   
 5 99 - 40630  1000 OCEAN AVE   New London 1000 OCEAN AVE New London CT  CT   
 6 100 - 41156 1001 OCEAN AVE   New London 1001 OCEAN AVE New London CT  CT   
 7 106 - 40981 1007 PEQUOT AVE  New London 1007 PEQUOT AVE New London CT CT   
 8 112 - 38326 101 COLMAN ST    New London 101 COLMAN ST New London CT   CT   
 9 118 - 40632 101 TRUMAN ST    New London 101 TRUMAN ST New London CT   CT   
10 120 - 40801 1011 BANK ST     New London 1011 BANK ST New London CT    CT   
# ℹ 516 more rows
# ℹ 6 more va

In [12]:
%%R

write_csv(data_geocoded, "New_London_geocoded.csv")

## Aquarion Water Co of CT-Greenwich

In [15]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Aquarion Water Co of CT-Greenwich.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 1552 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): STREET ADDRESS, TOWN, merged address, STATE, ENTIRE SERVICE LINE MA...
dbl (1): SITE ID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 9
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
      <dbl> <chr>            <chr> <chr>            <chr> <chr>                 
1 700152103 9 TINGUE ST      BYRAM 9 TINGUE ST BYR… CT    Lead                  
2 700164929 223 COGNEWAUGH … COS … 223 COGNEWAUGH … CT    Lead                  
3 700224686 48 VALLEY RD     COS … 48 VALLEY RD CO… CT    Lead                  
4 700148694 30 RIVER RD      COS … 30 RIVER RD COS… CT    Lead                  
5 700148819 8 MIAMI CT       COS … 8 MIAMI CT COS … CT    Lead                  
6 700148826 6 MIAMI CT       COS … 6 MIAMI CT

In [16]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

# A tibble: 6 × 11
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
      <dbl> <chr>            <chr> <chr>            <chr> <chr>                 
1 700152103 9 TINGUE ST      BYRAM 9 TINGUE ST BYR… CT    Lead                  
2 700164929 223 COGNEWAUGH … COS … 223 COGNEWAUGH … CT    Lead                  
3 700224686 48 VALLEY RD     COS … 48 VALLEY RD CO… CT    Lead                  
4 700148694 30 RIVER RD      COS … 30 RIVER RD COS… CT    Lead                  
5 700148819 8 MIAMI CT       COS … 8 MIAMI CT COS … CT    Lead                  
6 700148826 6 MIAMI CT       COS … 6 MIAMI CT COS … CT    Lead                  
# ℹ abbreviated name: ¹​`ENTIRE SERVICE LINE MATERIAL CLASSIFICATION`
# ℹ 5 more variables: `BUILDING TYPE` <chr>, PWSID <chr>, `SYSTEM NAME` <chr>,
#   lat <dbl>, long <dbl>


Passing 1,549 addresses to the Nominatim single address geocoder


In [17]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)

# A tibble: 1,552 × 11
   `SITE ID` `STREET ADDRESS`  TOWN    `merged address`             STATE
       <dbl> <chr>             <chr>   <chr>                        <chr>
 1 700152103 9 TINGUE ST       BYRAM   9 TINGUE ST BYRAM CT         CT   
 2 700164929 223 COGNEWAUGH RD COS COB 223 COGNEWAUGH RD COS COB CT CT   
 3 700224686 48 VALLEY RD      COS COB 48 VALLEY RD COS COB CT      CT   
 4 700148694 30 RIVER RD       COS COB 30 RIVER RD COS COB CT       CT   
 5 700148819 8 MIAMI CT        COS COB 8 MIAMI CT COS COB CT        CT   
 6 700148826 6 MIAMI CT        COS COB 6 MIAMI CT COS COB CT        CT   
 7 700148837 4 MEAD AVE        COS COB 4 MEAD AVE COS COB CT        CT   
 8 700148914 521 E PUTNAM AVE  COS COB 521 E PUTNAM AVE COS COB CT  CT   
 9 700148930 525 E PUTNAM AVE  COS COB 525 E PUTNAM AVE COS COB CT  CT   
10 700148936 531 PUTNAM AVE E  COS COB 531 PUTNAM AVE E COS COB CT  CT   
# ℹ 1,542 more rows
# ℹ 6 more variables: `ENTIRE SERVICE LINE MATERIAL CLASSIFICATION` <

In [18]:
%%R

write_csv(data_geocoded, "Aquarion_Greenwich_geocoded.csv")

## Middletown Water Department

In [19]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Middletown Water Department.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 257 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): SITE ID, STREET ADDRESS, TOWN, merged address, STATE, ENTIRE SERVIC...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 9
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
  <chr>     <chr>            <chr> <chr>            <chr> <chr>                 
1 00000007R 353 MAIN STREET  MIDD… 353 MAIN STREET… CT    Lead                  
2 00000032C 291 MAIN STREET  MIDD… 291 MAIN STREET… CT    Lead                  
3 00000042R 241 COURT STREET MIDD… 241 COURT STREE… CT    Lead                  
4 00000050R 346 MAIN STREET  MIDD… 346 MAIN STREET… CT    Lead                  
5 00000071R 318 MAIN STREET  MIDD… 318 MAIN STREET… CT    Lead                  
6 00000093R 15 PORTLAND STR… MIDD… 15 PORTLAND STR… CT    Lead 

In [20]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

# A tibble: 6 × 11
  `SITE ID` `STREET ADDRESS` TOWN  `merged address` STATE ENTIRE SERVICE LINE …¹
  <chr>     <chr>            <chr> <chr>            <chr> <chr>                 
1 00000007R 353 MAIN STREET  MIDD… 353 MAIN STREET… CT    Lead                  
2 00000032C 291 MAIN STREET  MIDD… 291 MAIN STREET… CT    Lead                  
3 00000042R 241 COURT STREET MIDD… 241 COURT STREE… CT    Lead                  
4 00000050R 346 MAIN STREET  MIDD… 346 MAIN STREET… CT    Lead                  
5 00000071R 318 MAIN STREET  MIDD… 318 MAIN STREET… CT    Lead                  
6 00000093R 15 PORTLAND STR… MIDD… 15 PORTLAND STR… CT    Lead                  
# ℹ abbreviated name: ¹​`ENTIRE SERVICE LINE MATERIAL CLASSIFICATION`
# ℹ 5 more variables: `BUILDING TYPE` <chr>, PWSID <chr>, `SYSTEM NAME` <chr>,
#   lat <dbl>, long <dbl>


Passing 257 addresses to the Nominatim single address geocoder


In [21]:
%%R


# Display the first few rows in the console
head(data_geocoded)

# Or print the entire data frame (if it's not too large)
print(data_geocoded)

# A tibble: 257 × 11
   `SITE ID` `STREET ADDRESS`   TOWN       `merged address`                STATE
   <chr>     <chr>              <chr>      <chr>                           <chr>
 1 00000007R 353 MAIN STREET    MIDDLETOWN 353 MAIN STREET MIDDLETOWN CT   CT   
 2 00000032C 291 MAIN STREET    MIDDLETOWN 291 MAIN STREET MIDDLETOWN CT   CT   
 3 00000042R 241 COURT STREET   MIDDLETOWN 241 COURT STREET MIDDLETOWN CT  CT   
 4 00000050R 346 MAIN STREET    MIDDLETOWN 346 MAIN STREET MIDDLETOWN CT   CT   
 5 00000071R 318 MAIN STREET    MIDDLETOWN 318 MAIN STREET MIDDLETOWN CT   CT   
 6 00000093R 15 PORTLAND STREET MIDDLETOWN 15 PORTLAND STREET MIDDLETOWN … CT   
 7 00000095R 13 LIBERTY STREET  MIDDLETOWN 13 LIBERTY STREET MIDDLETOWN CT CT   
 8 00000102R 111 RESERVOIR ROAD MIDDLETOWN 111 RESERVOIR ROAD MIDDLETOWN … CT   
 9 00000156R 430 BOSTON ROAD    MIDDLETOWN 430 BOSTON ROAD MIDDLETOWN CT   CT   
10 00000165R 186 COLLEGE STREET MIDDLETOWN 186 COLLEGE STREET MIDDLETOWN … CT   
# ℹ 247

In [22]:
%%R

write_csv(data_geocoded, "Middletown_geocoded.csv")

## Meriden Water Division

In [23]:
%%R

# Read in your CSV file (adjust the path if needed)
data <- read_csv("Meriden Water Division.csv")

# Preview the data to check the column names and structure
print(head(data))

Rows: 17245 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): LOCATION IDENTIFIER, STREET ADDRESS, TOWN, STATE, ENTIRE SERVICE L...
dbl  (1): SITE ID
lgl  (3): STREET NUMBER, STREET NAME, is_match

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 6 × 14
  `SITE ID` `LOCATION IDENTIFIER`    `STREET ADDRESS` TOWN    STATE
      <dbl> <chr>                    <chr>            <chr>   <chr>
1    113440 41.52868569,-72.78358041 84 DEXTER AVE    MERIDEN CT   
2    113442 41.54866818,-72.79577167 96 WILCOX AVE    MERIDEN CT   
3    113444 41.53827141,-72.81606451 62 BRADLEY AVE   MERIDEN CT   
4    113446 41.54638539,-72.82102614 70 CORRIGAN AVE  MERIDEN CT   
5    113448 41.52092511,-72.82961438 42 MEADOW ST     MERIDEN CT   
6    113450 41.52066789,-72.82489188 11 MCKENZIE AVE  MERIDEN CT   
# ℹ 9 more variab

In [24]:
%%R


# Geocode each row based on the 'merged address' column
data_geocoded <- data %>%
  geocode(address = `merged address`, method = "osm")

# Check the results
head(data_geocoded)

Error: "merged address" is not a column name in the input dataset.


RInterpreterError: Failed to parse and evaluate line '\n\n# Geocode each row based on the \'merged address\' column\ndata_geocoded <- data %>%\n  geocode(address = `merged address`, method = "osm")\n\n# Check the results\nhead(data_geocoded)\n'.
R error message: 'Error: "merged address" is not a column name in the input dataset.'